In [79]:
import pandas as pd
import os
import time
import logging
from datetime import date

In [80]:
from googletrans import Translator

In [81]:
# # 获取脚本的当前路径，避免计划执行时路径出错
# home_dir = os.path.dirname(os.path.realpath(__file__))
# # 更换workding directory
# working_directory = home_dir
# os.chdir(working_directory)

In [82]:
proxy = 'http://127.0.0.1:1080'
os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [83]:
# translator = Translator(service_urls=[
#       'translate.google.com',
#       'translate.google.co.kr',
#     ])
# translator.translate('韩国', dest='ko').text

In [84]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [85]:
folder_path = '../shopee_product_translation/'
tw_site_product_list = pd.read_excel(os.path.join(folder_path, 'tengus_tw_product_list.xlsx'))
# tw_site_product_list = pd.read_excel(os.path.join(folder_path, 'translated_product_2019_10_19.xlsx'))
# tw_site_product_list = tw_site_product_list[tw_site_product_list['是否翻译'] == 'no']
# tw_site_product_list.head(5)

In [86]:
# tw_site_product_list.columns

In [87]:
# tw_site# 获取脚本的当前路径，避免计划执行时路径出错
# home_dir = os.path.dirname(os.path.realpath(__file__))
# # 更换workding directory
# working_directory = home_dir
# os.chdir(working_directory)_product_list = tw_site_product_list[tw_site_product_list['产品id'] == 2496637042]

In [88]:
# top_cat_in_tw = tw_site_product_list[['分类ID', '产品id']].groupby('分类ID').agg({'产品id': 'nunique'}).reset_index().sort_values(by='产品id', ascending=False).reset_index().head(5)
# top_cat_in_tw

In [89]:
# top_cat_id = top_cat_in_tw['分类ID'][0]
# top_cat_id

In [90]:
output_excel_columns = [
                        '变种名称',
                        '变种属性名称一',
                        '变种属性名称二',
                        '变种属性值一',
                        '变种属性值二',
                        '*价格（必填）',
                        '促销价',
                        '折扣活动ID',
                        '*库存（必填）',
                        '*重量(kg)(必填）',
                        '*主图（URL）地址（必填）',
                        '附图1',
                        '附图2',
                        '附图3',
                        '附图4',
                        '附图5',
                        '附图6',
                        '附图7',
                        '附图8',
                        '变种图',
                        '长(cm)',
                        '宽(cm)',
                        '高(cm)',
                        '发货期',
                        '来源URL',
                        '尺码图'
]

In [91]:
tw_site_product_list_selected_columns = ['分类ID', '产品属性', 'Parent SKU', '产品标题', '产品描述', 'sku', '变种名称', '变种属性名称一',
       '变种属性名称二', '变种属性值一', '变种属性值二', '价格', '促销价', '折扣活动ID', '库存', '重量', '主图（URL）地址', '附图1',
       '附图2', '附图3', '附图4', '附图5', '附图6', '附图7', '附图8地址', '变种图', '长（cm）',
       '宽（cm）', '高（cm）', '发货期', '来源URL', '尺码图']

In [92]:
tw_site_product_list_with_selected_columns = tw_site_product_list[tw_site_product_list_selected_columns]
# tw_site_product_list_with_selected_columns.head(5)

In [93]:
# tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns[tw_site_product_list_with_selected_columns['分类ID'] == top_cat_id].reset_index()
# tw_site_product_list_with_selected_columns

In [94]:
# 把1688的来源url拆出来，把productid填充为parent sku
tw_site_product_list_with_selected_columns['Parent SKU'] = tw_site_product_list_with_selected_columns['来源URL'].str.split("/", n = 4, expand = True)[4].str.split(".", n = 1, expand=True)[0]
# tw_site_product_list_with_selected_columns.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [95]:
# remove emoji
import re

emoji_pattern = re.compile(
                u'['
                u'\U0001F300-\U0001F64F'
                u'\U0001F680-\U0001F6FF'
                u'\u2600-\u2B55'
                u'\u23cf'
                u'\u23e9'
                u'\u231a'
                u'\u3030'
                u'\ufe0f'
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u'\U00010000-\U0010ffff'
                u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
                u'\U00002702-\U000027B0]+', flags=re.UNICODE)

def remove_emoji(text):
    return emoji_pattern.sub(r'', text)

# remove_emoji('ins爆款 🥕胡蘿卜🥕寶寶針織竹節棉爬服 嬰兒兔裝 嬰兒連身衣 寶寶包屁衣連身衣 中間開襟包屁衣 寶寶連體衣哈衣爬服')

In [96]:
tw_site_product_list_with_selected_columns['产品标题'] = [remove_emoji(x) for x in tw_site_product_list_with_selected_columns['产品标题']]
tw_site_product_list_with_selected_columns['产品描述'] = [remove_emoji(x) for x in tw_site_product_list_with_selected_columns['产品描述']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [97]:
tw_site_product_list_with_selected_columns['长（cm）'] = tw_site_product_list_with_selected_columns['长（cm）'].fillna(18)
tw_site_product_list_with_selected_columns['宽（cm）'] = tw_site_product_list_with_selected_columns['宽（cm）'].fillna(13)
tw_site_product_list_with_selected_columns['高（cm）'] = tw_site_product_list_with_selected_columns['高（cm）'].fillna(5)
tw_site_product_list_with_selected_columns['发货期'] = tw_site_product_list_with_selected_columns['发货期'].fillna(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [98]:
set(tw_site_product_list_with_selected_columns['变种属性名称一'].tolist())

{'颜色'}

In [99]:
set(tw_site_product_list_with_selected_columns['变种属性名称二'].tolist())

{'适合身高'}

In [100]:
set(tw_site_product_list_with_selected_columns['变种属性值一'].tolist())

{'2DTZ9276B蓝碎花',
 '2DTZ9276B蓝色',
 '2DTZ9277Z紫碎花',
 '2DTZ9277Z紫色',
 '2DTZ9277紫色',
 '2DTZ9278A灰碎花',
 '2DTZ9278A灰色',
 '2DTZ9278灰色',
 '2DTZ9279W灰色花朵',
 '2DTZ9281G绿色',
 '2DTZ9282Rr玫红色',
 '2DTZ9283 粉色',
 '2DTZ9283P粉色',
 '2DTZ9287B',
 '2DTZ9287B条纹',
 '2DTZ9287蓝色',
 '2DTZ9288Y黄色',
 '2DTZ9288黄色条纹',
 '2DTZ9304Y灰+黄',
 '2DTZ9304黄色',
 '2DTZ9308W白色',
 '2DTZ9309G字母',
 '2DTZ9309G绿色',
 '2KT5007G爱心',
 '2KT5008G徽章',
 '2KT5010Y蝴蝶',
 '2KT5011Y字母',
 '3DTZ9284  helloworld+条纹花朵裤',
 '3DTZ9284Whelloworld+条纹花朵裤',
 '3DTZ9286Whelloworld+黑色鹿角裤',
 '3DTZ9286helloworld+黑色鹿角裤',
 '3DTZ9289B带鹿角条纹款',
 '3DTZ9291W黄色',
 '3DTZ9292B',
 '3DTZ9292B蓝色',
 '3DTZ9292藏蓝色',
 '3DTZ9294B蓝色',
 '3DTZ9294蓝色',
 '3DTZ9296A灰色',
 '3DTZ9297W麋鹿',
 '3DTZ9298 HELLO',
 '3DTZ9298W',
 '3DTZ9298W Hello World三件套',
 '3DTZ9310  绿色3件套',
 '3DTZ9310G绿色碎花',
 '3XTZ8215W',
 '3XTZ8215W条纹',
 '3XTZ8217W军绿三件套',
 '3XTZ8240N',
 '4XTZ8230W斑马纹',
 '8221白色',
 '8221黑色',
 'DTZ9038A小汽车套装',
 'DTZ9112A灰色',
 'DTZ9112W白色',
 'DTZ9157A灰色',
 'DTZ9158A灰色',
 'DTZ9160W猫头鹰',
 'DTZ916

In [101]:
set(tw_site_product_list_with_selected_columns['变种属性值二'].tolist())

{'100',
 '100cm',
 '11',
 '110',
 '110cm',
 '120',
 '120cm',
 '12M',
 '130',
 '130cm',
 '140',
 '140cm',
 '150',
 '150cm',
 '18-24M',
 '18M',
 '2',
 '24M',
 '2T',
 '2Y',
 '3',
 '3T',
 '3Y',
 '4',
 '4T',
 '4Y',
 '5',
 '59',
 '5T',
 '5Y',
 '6',
 '60cm',
 '66',
 '6T',
 '6Y',
 '7',
 '70',
 '70cm',
 '73',
 '7T',
 '8',
 '80',
 '80cm',
 '85',
 '8T',
 '9',
 '90',
 '90cm',
 '95',
 '95CM',
 '95cm',
 '9M',
 'L',
 'M',
 'S',
 'XL',
 'XXL'}

In [102]:
# translate_currency = 458.61
# profit_rate = 1.2
# tw_site_product_list_with_selected_columns['价格'] = tw_site_product_list_with_selected_columns['价格'] * translate_currency / profit_rate

In [103]:
# tw_site_product_list_with_selected_columns.head(5)

In [104]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
    ])
# translator.translate('韩国', dest='ko').text

In [105]:
# translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='ko')
# for translation in translations:
#     print(translation.origin, ' -> ', translation.text)

In [106]:
translate_columns = ['产品标题', '产品描述', '变种名称', '变种属性名称一', '变种属性名称二', '变种属性值一', '变种属性值二']
# translate_columns = ['产品标题']
# translate_columns = ['产品描述', '变种名称', '变种属性名称一', '变种属性名称二', '变种属性值一', '变种属性值二']

In [107]:
# 把空值填充为string格式
tw_site_product_list_with_selected_columns[translate_columns] = tw_site_product_list_with_selected_columns[translate_columns].fillna(value='不需翻译')

In [108]:
# 把variation_name做一些删减
variation_list = set(list(tw_site_product_list_with_selected_columns['变种属性名称一'].unique()) + list(tw_site_product_list_with_selected_columns['变种属性名称二'].unique()))
logging.info('variation list: ' + str(variation_list))

 2019-11-30 13:20:52,799 - INFO - variation list: {'适合身高', '颜色'}


In [109]:
variation_rename = {'适合身高': '身高',
                    '童袜尺码': '尺码',
                    '不需翻译': 'not set',
                    '73CM': '73cm',
                    '100cm 少量现货': '100cm',
                    '100CM建议体重32斤内': '100cm',
                    '110CM建议体重36斤内': '110cm',
                    '120CM建议体重40斤内': '120cm',
                    '73CM建议体重18斤内': '73cm',
                    '80CM建议体重23斤内': '80cm',
                    '90CM建议体重28斤内': '90cm',
                    '编码': 'no size',
                    'XXL码（建议身高130-150cm）':'XXL',
                    'L码 （建议身高100-100cm）':'L ',
                    'XL码(建议身高115-130cm)':'XL',
                    '20/21（实际内长12.4cm）':'20/21',
                    '22/23（实际内长13.6cm）':'22/23',
                    '24/25（实际内长14.7cm）':'24/25',
                    '26/27（实际内长15.9CM）':'26/27',
                    '28/29（实际内长17cm）':'28/29',
                    '袋装（加大 6-18个月）':'6-18月',
                    '袋装（常规 0-12个月）':'0-12月',
                    'M码（2-4岁 防滑底）':'M',
                    'S码（0-2岁 防滑底）':'S',
                    'M（1-3岁 防滑底）':'M',
                    'M（2-4岁 防滑底）':'M',
                    'S（0-1岁 防滑底）':'S',
                    'S（0-2岁 防滑底）':'S',
                    'S (6-12)个月':'S ',
                    'S （6-12个月）':'S ',
                    'XS( 0-6个)月':'XS',
                    '12-24个月闭档':'12-24月',
                    '32*12.5cm':'32*12.5cm',
                    'L码24-36个月':'L',
                    'XL码 9-12岁':'XL',
                    'XL码（5-8岁）':'XL',
                    'XS（0-6个月）':'XS',
                    '6-12个月闭档':'6-12月',
                    'L码 7-10岁':'L',
                    'L码（3-5岁）':'L',
                    'L码（7-9岁）':'L',
                    'M （1-3）岁':'M ',
                    'M码（1-3岁）':'M',
                    'M码（4-6岁）':'M',
                    'S码（0-1岁）':'S',
                    'S码（1-3岁）':'S',
                    'XL码 5-8岁':'XL',
                    'XL码9-12岁':'XL',
                    '（0-1岁）均码':'0-1岁',
                    'L码 3-5岁':'L',
                    'L码 5-8岁':'L',
                    'L（3-5岁）':'L',
                    'M码 1-3岁':'M',
                    'M码 3-5岁':'M',
                    'M码 4-6岁':'M',
                    'M（1-3岁）':'M',
                    'S码 0-1岁':'S',
                    'S码 1-3岁':'S',
                    '（1-3岁）均码':'1-3岁',
                    '12*9CM':'12*9CM',
                    '12-24月':'12-24月',
                    '2-4岁闭档':'2-4岁',
                    '4-6岁闭档':'4-6岁',
                    '6-12个月':'6-12月',
                    '6-18个月':'6-18月',
                    'L码3-5岁':'L',
                    'L码4-6岁':'L',
                    'L码5-8岁':'L',
                    'L码6-8岁':'L',
                    'M码1-3岁':'M',
                    'M码2-4岁':'M',
                    'M码3-5岁':'M',
                    'S码0-1岁':'S',
                    'S码0-2岁':'S',
                    'S码1-3岁':'S',
                    '0-2岁':'0-2岁',
                    '1-3岁':'1-3岁',
                    '11CM':'11CM',
                    '11cm':'11cm',
                    '12CM':'12CM',
                    '12cm':'12cm',
                    '13CM':'13CM',
                    '13cm':'13cm',
                    '14cm':'14cm',
                    '2-4岁':'2-4岁',
                    '28CM':'28CM',
                    '28cm':'28cm',
                    '3-5岁':'3-5岁',
                    '35CM':'35CM',
                    '35cm':'35cm',
                    '4-6岁':'4-6岁',
                    '45CM':'45CM',
                    '45cm':'45cm',
                    '5-8岁':'5-8岁',
                    '均码':'均码'
}
tw_site_product_list_with_selected_columns['变种属性名称一'] = tw_site_product_list_with_selected_columns['变种属性名称一'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性名称二'] = tw_site_product_list_with_selected_columns['变种属性名称二'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性值一'] = tw_site_product_list_with_selected_columns['变种属性值一'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性值二'] = tw_site_product_list_with_selected_columns['变种属性值二'].apply(lambda x: variation_rename[x] if x in variation_rename else x)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [110]:
# 给变种属性值一、二做适当缩减
# 先把索引列取出来
tw_site_product_list_with_selected_columns['index'] = tw_site_product_list_with_selected_columns.index
tw_site_product_list_with_selected_columns['变种属性值一'] = tw_site_product_list_with_selected_columns['index'].groupby([tw_site_product_list_with_selected_columns['Parent SKU'],tw_site_product_list_with_selected_columns['变种属性值二']]).rank(ascending=1,method='dense')

tw_site_product_list_with_selected_columns['变种属性值一'] = tw_site_product_list_with_selected_columns['变种属性值一'].apply(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [111]:
tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns.drop(['index'], axis=1)

In [112]:
# 价格 折扣价
tw_site_product_list_with_selected_columns['促销价'] = tw_site_product_list_with_selected_columns['价格']
tw_site_product_list_with_selected_columns['价格'] = tw_site_product_list_with_selected_columns['价格'] * 10
tw_site_product_list_with_selected_columns['折扣活动ID'] = 1035824326

# 重量
tw_site_product_list_with_selected_columns['重量'] = 0.3

In [113]:
tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'to_be_translated_product.xlsx'), index=False)

In [114]:
# 对variation_value做一些删减
# variation_value_list = set(list(tw_site_product_list_with_selected_columns['变种属性值一'].unique()) + list(tw_site_product_list_with_selected_columns['变种属性值一'].unique()))
# logging.info('variation value list: ' + str(variation_value_list))

In [115]:
# tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns[:5]
# # tw_site_product_list_with_selected_columns

In [116]:
today = date.today()
d1 = today.strftime("%Y_%m_%d")

In [117]:
# for index, row in tw_site_product_list_with_selected_columns.iterrows():
#     try:
#         list_to_translate = []
#         for i, value in enumerate(translate_columns):
#             list_to_translate.append(row[translate_columns[i]])
        
#         translate_language = 'en'
#         translator = Translator(service_urls=['translate.google.com'])
#         list_be_translated = translator.translate(list_to_translate, dest=translate_language, src='zh-CN')
#         # list_be_translated = translator.translate(list_to_translate, dest=translate_language)
#         time.sleep(10)
        
#         i = 0
#         for translate_language in list_be_translated:
#             tw_site_product_list_with_selected_columns.at[index, translate_columns[i]] = translate_language.text
#             i += 1
        
#         if (index + 1) % 5 == 0:
#             logging.info(str(index + 1) + ' rows have been translated.')
            
#         tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'translated_product_' + d1 + '.xlsx'), index=False)
#     except Exception as err:
#         logging.warning(str(index + 1) + ' rows have not been translated. Error message: ' + str(err))
#         pass

In [118]:
for index, row in tw_site_product_list_with_selected_columns.iterrows():
    translated = 0
    while translated == 0:
        try:
            list_to_translate = []
            for i, value in enumerate(translate_columns):
                list_to_translate.append(row[translate_columns[i]])

            # translate_language = 'id'
            translate_language = 'th'
            translator = Translator(service_urls=['translate.google.com'])
            list_be_translated = translator.translate(list_to_translate, dest=translate_language, src='zh-CN')
            # list_be_translated = translator.translate(list_to_translate, dest=translate_language)
            time.sleep(10)

            i = 0
            for translate_language in list_be_translated:
                tw_site_product_list_with_selected_columns.at[index, translate_columns[i]] = translate_language.text
                i += 1

            if (index + 1) % 100 == 0:
                logging.info(str(index + 1) + ' rows have been translated.')

            tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'translated_product_th_' + d1 + '.xlsx'), index=False)
            translated = 1
        except Exception as err:
            logging.warning(str(index + 1) + ' rows have not been translated. Error message: ' + str(err) + ', try again.')
            pass        

 2019-11-30 13:54:40,890 - INFO - 100 rows have been translated.
 2019-11-30 14:28:11,844 - INFO - 200 rows have been translated.
 2019-11-30 15:01:10,875 - INFO - 300 rows have been translated.
 2019-11-30 15:34:31,468 - INFO - 400 rows have been translated.
 2019-11-30 16:07:28,103 - INFO - 500 rows have been translated.
 2019-11-30 16:40:19,218 - INFO - 600 rows have been translated.
 2019-11-30 17:13:15,797 - INFO - 700 rows have been translated.
 2019-11-30 17:24:20,281 - WARNING - 734 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 17:46:11,640 - INFO - 800 rows have been translated.
 2019-11-30 18:19:00,218 - INFO - 900 rows have been translated.
 2019-11-30 18:51:34,609 - INFO - 1000 rows have been translated.
 2019-11-30 19:24:17,406 - INFO - 1100 rows have been translated.
 2019-1

 2019-11-30 19:28:17,699 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:17,734 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:17,765 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:17,812 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-11-30 19:28:18,991 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:19,055 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:19,117 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:19,156 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-11-30 19:28:20,067 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:20,094 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:20,126 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:20,148 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-11-30 19:28:20,800 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:20,824 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:20,851 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-30 19:28:20,875 - WARNING - 1113 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

...

 2019-12-01 09:09:19,965 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:19,996 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:20,023 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:20,058 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-12-01 09:09:20,851 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:20,878 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:20,906 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:20,925 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-12-01 09:09:21,699 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:21,726 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:21,757 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:21,789 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-12-01 09:09:22,476 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:22,511 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:22,539 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-01 09:09:22,574 - WARNING - 3588 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-12-01 18:10:03,891 - INFO - 5200 rows have been translated.
 2019-12-01 18:43:06,221 - INFO - 5300 rows have been translated.


In [119]:
# 优化结果
output_path = os.path.join(folder_path, 'translated_product_th_' + d1 + '.xlsx')
dianxiaomi_product_list = pd.read_excel(output_path)

dianxiaomi_product_list['产品标题'] = dianxiaomi_product_list['产品标题'].apply(lambda x: x[:100])

variation_name_key_value = {
    'ไม่จำเป็นต้องแปล': 'size',
    'ไม่ได้ตั้งค่า': 'size',
    'เหมาะสำหรับความสูง': 'ความสูง',
    'ข้อมูลจำเพาะ (กว้าง * ยาวซม.)': 'size',
    'tidak diatur': 'size'
}

dianxiaomi_product_list['变种属性名称二'] = dianxiaomi_product_list['变种属性名称二'].apply(lambda x: variation_name_key_value[x] if x in variation_name_key_value else x)

In [120]:
# dianxiaomi_product_list['index'] = dianxiaomi_product_list.index
# dianxiaomi_product_list['变种属性值一'] = dianxiaomi_product_list['index'].groupby([dianxiaomi_product_list['Parent SKU'],dianxiaomi_product_list['变种属性值二']]).rank(ascending=1,method='dense')

# dianxiaomi_product_list['变种属性值一'] = dianxiaomi_product_list['变种属性值一'].apply(str)

In [121]:
dianxiaomi_product_list.to_excel(os.path.join(folder_path, 'translated_product_th_' + d1 + '_edited(1).xlsx'), index=False)

In [122]:
# for translate_column in translate_columns:
#     # tw_site_product_list_with_selected_columns[translate_column] = tw_site_product_list_with_selected_columns[translate_column].fillna(0)
#     # tw_site_product_list_with_selected_columns[translate_column]  = [translator.translate(x, dest=translate_language).text 
#                                                                      # for x in tw_site_product_list_with_selected_columns[translate_column]]
#     for index, row in tw_site_product_list_with_selected_columns.iterrows():
#         try:
#             orginal_text = tw_site_product_list_with_selected_columns.at[index, translate_column]
#             tw_site_product_list_with_selected_columns.at[index, translate_column] = translator.translate(row[translate_column], dest=translate_language).text
#             # print(row[translate_column])
#             logging.info(translate_column + ' - ' + str(index + 1) + ' rows have been translated.')
#             # logging.info(orginal_text + ' -> ' + tw_site_product_list_with_selected_columns.at[index, translate_column])
#             time.sleep(5)
#         except:
#             logging.warning(translate_column + ' - ' + str(index + 1) + ' rows have not been translated.')
#             pass
        
#     # print(tw_site_product_list_with_selected_columns.head(5))
    
#     tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'tw_translated_product(1).xlsx'), index=False)
#     logging.info(str(translate_column) + ' is done.')